In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize
from heapq import nsmallest
from tqdm import tqdm_notebook

In [2]:
json_data=open('/home/laptop/Downloads/vet_uk.json').read()
data = json.loads(json_data)
final_data = json_normalize(data['elements'])

In [3]:
final_data.head()

,id,lat,lon,tags.accuracy,tags.addr:city,tags.addr:country,tags.addr:county,tags.addr:housename,tags.addr:housenumber,tags.addr:interpolation,...,tags.toilets:wheelchair,tags.uri,tags.url,tags.veterinary,tags.website,tags.wheelchair,tags.wheelchair:description,tags.wikidata,tags.wikipedia,type
0,30257157,51.908494,-1.168052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,node
1,31437643,51.560862,-0.358228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,node
2,101386320,53.340885,-3.003925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,node
3,206360557,51.354638,-1.153140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,node
4,207558859,51.357017,-1.140848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,node


In [4]:
ukpostcode = pd.read_csv('/home/laptop/Downloads/ukpostcodes.csv')

In [5]:
ukpostcode.head()

,id,postcode,latitude,longitude
0,1,AB10 1XG,57.144165,-2.114848
1,2,AB10 6RN,57.137880,-2.121487
2,3,AB10 7JB,57.124274,-2.127190
3,4,AB11 5QN,57.142701,-2.093295
4,5,AB11 6UL,57.137547,-2.112233


# Join

In [17]:
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))


In [7]:
tempDataList = final_data[['lat', 'lon']].to_dict('records')

In [8]:
ukpostcode.rename(index=str, columns={"latitude": "lat", "longitude": "lon"}, inplace=True)
ukpostcode_to_ditc = ukpostcode[['lat', 'lon']].to_dict('records')

In [20]:
vet_lat_1 = []
vet_lon_1 = []
vet_lat_2 = []
vet_lon_2 = []
LOG_EVERY_N = 10000
for i in tqdm_notebook(range(ukpostcode.shape[0])):
    if (i % LOG_EVERY_N) == 0:
        print(i)
    closest_nb1 = closest(tempDataList, ukpostcode_to_ditc[i])
    vet_lat_1.append(closest_nb1['lat'])
    vet_lon_1.append(closest_nb1['lon'])

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

In [21]:
ukpostcode['vet_lat1'] = vet_lat_1
ukpostcode['vet_lon1'] = vet_lon_1

In [23]:
import numpy as np

In [26]:
vdistance = np.vectorize(distance)

In [27]:
ukpostcode['distance_vet1'] = vdistance(ukpostcode['lat'], ukpostcode['lon'], ukpostcode['vet_lat1'], ukpostcode['vet_lon1'])

In [31]:
ukpostcode.head()

,id,postcode,lat,lon,vet_lat1,vet_lon1,distance_vet1
0,1,AB10 1XG,57.144165,-2.114848,57.155417,-2.106702,1.344172
1,2,AB10 6RN,57.137880,-2.121487,57.155417,-2.106702,2.144326
2,3,AB10 7JB,57.124274,-2.127190,57.155417,-2.106702,3.676972
3,4,AB11 5QN,57.142701,-2.093295,57.155417,-2.106702,1.628857
4,5,AB11 6UL,57.137547,-2.112233,57.155417,-2.106702,2.014914


In [34]:
ukpostcode_small = ukpostcode.drop(['id', 'lat', 'lon'], axis=1)

In [33]:
ukpostcode.to_csv('/home/laptop/Downloads/vet_uk_dl.csv', index=False)

In [35]:
ukpostcode_small.to_csv('/home/laptop/Downloads/vet_uk_dl_small.csv', index=False)